# Zenith Insurance ML Models - Model Registry

This notebook trains ML models for the Zenith Insurance Intelligence Agent:
- **Claim Cost Prediction** - Predict total incurred costs for claims
- **Fraud Detection** - Classify claims as potential fraud risks
- **Return-to-Work Timeline** - Predict days to return to work

All models are registered to Snowflake Model Registry and can be added as tools to the Intelligence Agent.

## Before You Begin

**Add these packages** in the Packages dropdown (upper right):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database:** ZENITH_INSURANCE_INTELLIGENCE  
**Schema:** ANALYTICS  
**Warehouse:** ZENITH_WH


## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get active session and set context to Zenith Insurance database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('ZENITH_INSURANCE_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('ZENITH_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Claim Cost Prediction

Predict total incurred costs for claims based on injury characteristics.


### Prepare Claim Cost Training Data


In [ ]:
# Get claim data with features
claim_cost_df = session.sql("""
SELECT
    c.claim_id,
    c.injury_type,
    c.body_part,
    c.claim_type,
    c.severity,
    e.industry_vertical,
    e.business_segment,
    iw.age::FLOAT AS worker_age,
    iw.years_of_experience::FLOAT AS worker_experience,
    iw.safety_training_completed::BOOLEAN AS safety_trained,
    c.days_lost::FLOAT AS days_lost,
    c.total_incurred::FLOAT AS total_incurred
FROM RAW.CLAIMS c
JOIN RAW.EMPLOYERS e ON c.employer_id = e.employer_id
LEFT JOIN RAW.INJURED_WORKERS iw ON c.injured_worker_id = iw.injured_worker_id
WHERE c.claim_status = 'CLOSED'
  AND c.total_incurred > 0
  AND c.injury_date >= DATEADD('year', -3, CURRENT_DATE())
""")

print(f"Claim cost data: {claim_cost_df.count()} closed claims")
claim_cost_df.show(5)


### Train Claim Cost Model


In [ ]:
# Train/test split
train_cost, test_cost = claim_cost_df.random_split([0.8, 0.2], seed=42)

# Drop CLAIM_ID (VARCHAR not supported)
train_cost = train_cost.drop("CLAIM_ID")
test_cost = test_cost.drop("CLAIM_ID")

# Create pipeline
cost_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["INJURY_TYPE", "BODY_PART", "CLAIM_TYPE", "SEVERITY", "INDUSTRY_VERTICAL", "BUSINESS_SEGMENT"],
        output_cols=["INJURY_TYPE_ENC", "BODY_PART_ENC", "CLAIM_TYPE_ENC", "SEVERITY_ENC", "INDUSTRY_ENC", "SEGMENT_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["WORKER_AGE", "WORKER_EXPERIENCE", "DAYS_LOST"],
        output_cols=["WORKER_AGE_SCALED", "WORKER_EXPERIENCE_SCALED", "DAYS_LOST_SCALED"]
    )),
    ("Regressor", LinearRegression(
        label_cols=["TOTAL_INCURRED"],
        output_cols=["PREDICTED_COST"]
    ))
])

# Train model
cost_pipeline.fit(train_cost)
print("✅ Claim cost prediction model trained")


### Evaluate and Register Claim Cost Model

In [ ]:
# Make predictions on test set
cost_predictions = cost_pipeline.predict(test_cost)

# Calculate metrics
mae = mean_absolute_error(df=cost_predictions, y_true_col_names="TOTAL_INCURRED", y_pred_col_names="PREDICTED_COST")
mse = mean_squared_error(df=cost_predictions, y_true_col_names="TOTAL_INCURRED", y_pred_col_names="PREDICTED_COST")
rmse = mse ** 0.5

cost_metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Claim cost model metrics: {cost_metrics}")

# Register model to Model Registry
reg = Registry(session)
reg.log_model(
    model=cost_pipeline,
    model_name="CLAIM_COST_PREDICTOR",
    version_name="V1",
    comment="Predicts total incurred claim costs using Linear Regression based on injury characteristics and worker demographics",
    metrics=cost_metrics
)

print("✅ Claim cost model registered to Model Registry as CLAIM_COST_PREDICTOR")


---
# MODEL 2: Fraud Detection

Classify claims as potential fraud risks based on claim characteristics and patterns.


### Prepare Fraud Detection Training Data


In [ ]:
# Get claim features for fraud detection
fraud_df = session.sql("""
SELECT
    c.claim_id,
    c.injury_type,
    c.body_part,
    c.claim_type,
    c.severity,
    e.industry_vertical,
    -- Days between injury and report (red flag if long delay)
    DATEDIFF('day', c.injury_date, c.report_date)::FLOAT AS days_to_report,
    c.litigated::BOOLEAN AS has_litigation,
    iw.injury_history_count::FLOAT AS prior_injuries,
    -- Cost ratio (high indemnity vs medical can indicate fraud)
    CASE WHEN c.medical_paid > 0 
         THEN (c.indemnity_paid / NULLIF(c.medical_paid, 0))::FLOAT 
         ELSE 0.0 END AS indemnity_medical_ratio,
    -- Target: Has SIU investigation or dispute
    (EXISTS (SELECT 1 FROM RAW.SIU_INVESTIGATION_REPORTS sir 
             WHERE sir.claim_id = c.claim_id 
             AND sir.investigation_status = 'CONFIRMED_FRAUD')
     OR EXISTS (SELECT 1 FROM RAW.CLAIM_DISPUTES cd 
                WHERE cd.claim_id = c.claim_id 
                AND cd.dispute_type IN ('COMPENSABILITY', 'CAUSATION')))::BOOLEAN AS is_fraud_risk
FROM RAW.CLAIMS c
JOIN RAW.EMPLOYERS e ON c.employer_id = e.employer_id
LEFT JOIN RAW.INJURED_WORKERS iw ON c.injured_worker_id = iw.injured_worker_id
WHERE c.claim_status IN ('CLOSED', 'LITIGATED')
  AND c.injury_date >= DATEADD('year', -3, CURRENT_DATE())
""")

print(f"Fraud detection data: {fraud_df.count()} claims")
fraud_df.show(5)


### Train Fraud Detection Model


In [ ]:
# Train/test split
train_fraud, test_fraud = fraud_df.random_split([0.8, 0.2], seed=42)

# Drop CLAIM_ID
train_fraud = train_fraud.drop("CLAIM_ID")
test_fraud = test_fraud.drop("CLAIM_ID")

# Create pipeline
fraud_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["INJURY_TYPE", "BODY_PART", "CLAIM_TYPE", "SEVERITY", "INDUSTRY_VERTICAL"],
        output_cols=["INJURY_TYPE_ENC", "BODY_PART_ENC", "CLAIM_TYPE_ENC", "SEVERITY_ENC", "INDUSTRY_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_FRAUD_RISK"],
        output_cols=["FRAUD_PREDICTION"],
        n_estimators=100,
        max_depth=15
    ))
])

# Train model
fraud_pipeline.fit(train_fraud)
print("✅ Fraud detection model trained")


### Evaluate and Register Fraud Mo

In [ ]:
# Make predictions
fraud_predictions = fraud_pipeline.predict(test_fraud)

# Calculate metrics
fraud_accuracy = accuracy_score(df=fraud_predictions, y_true_col_names="IS_FRAUD_RISK", y_pred_col_names="FRAUD_PREDICTION")
fraud_metrics = {"accuracy": round(fraud_accuracy, 4)}
print(f"Fraud detection model metrics: {fraud_metrics}")

# Register model
reg.log_model(
    model=fraud_pipeline,
    model_name="FRAUD_DETECTOR",
    version_name="V1",
    comment="Classifies claims as fraud risks using Random Forest based on claim patterns and anomalies",
    metrics=fraud_metrics
)

print("✅ Fraud detection model registered to Model Registry as FRAUD_DETECTOR")


---
# MODEL 3: Return-to-Work Timeline Prediction

Predict number of days until injured worker returns to work.


### Prepare Return-to-Work Training Data


In [ ]:
# Get return-to-work data
rtw_df = session.sql("""
SELECT
    c.claim_id,
    c.injury_type,
    c.body_part,
    c.claim_type,
    c.severity,
    e.industry_vertical,
    e.business_segment,
    e.safety_rating,
    iw.age::FLOAT AS worker_age,
    iw.years_of_experience::FLOAT AS worker_experience,
    iw.safety_training_completed::BOOLEAN AS safety_trained,
    adj.adjuster_type,
    -- Target: Days to return to work
    DATEDIFF('day', c.injury_date, c.return_to_work_date)::FLOAT AS days_to_rtw
FROM RAW.CLAIMS c
JOIN RAW.EMPLOYERS e ON c.employer_id = e.employer_id
LEFT JOIN RAW.INJURED_WORKERS iw ON c.injured_worker_id = iw.injured_worker_id
LEFT JOIN RAW.CLAIMS_ADJUSTERS adj ON c.adjuster_id = adj.adjuster_id
WHERE c.return_to_work_date IS NOT NULL
  AND c.claim_status = 'CLOSED'
  AND DATEDIFF('day', c.injury_date, c.return_to_work_date) BETWEEN 1 AND 365
""")

print(f"Return-to-work data: {rtw_df.count()} claims with RTW")
rtw_df.show(5)


### Train Return-to-Work Model


In [ ]:
# Train/test split
train_rtw, test_rtw = rtw_df.random_split([0.8, 0.2], seed=42)

# Drop CLAIM_ID
train_rtw = train_rtw.drop("CLAIM_ID")
test_rtw = test_rtw.drop("CLAIM_ID")

# Create pipeline
rtw_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["INJURY_TYPE", "BODY_PART", "CLAIM_TYPE", "SEVERITY", 
                   "INDUSTRY_VERTICAL", "BUSINESS_SEGMENT", "SAFETY_RATING", "ADJUSTER_TYPE"],
        output_cols=["INJURY_TYPE_ENC", "BODY_PART_ENC", "CLAIM_TYPE_ENC", "SEVERITY_ENC",
                    "INDUSTRY_ENC", "SEGMENT_ENC", "SAFETY_ENC", "ADJUSTER_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["WORKER_AGE", "WORKER_EXPERIENCE"],
        output_cols=["WORKER_AGE_SCALED", "WORKER_EXPERIENCE_SCALED"]
    )),
    ("Regressor", LinearRegression(
        label_cols=["DAYS_TO_RTW"],
        output_cols=["PREDICTED_DAYS_TO_RTW"]
    ))
])

# Train
rtw_pipeline.fit(train_rtw)
print("✅ Return-to-work timeline model trained")


### Evaluate and Register RTW Model


In [ ]:
# Predict on test set
rtw_predictions = rtw_pipeline.predict(test_rtw)

# Calculate accuracy
rtw_mae = mean_absolute_error(df=rtw_predictions, y_true_col_names="DAYS_TO_RTW", y_pred_col_names="PREDICTED_DAYS_TO_RTW")
rtw_mse = mean_squared_error(df=rtw_predictions, y_true_col_names="DAYS_TO_RTW", y_pred_col_names="PREDICTED_DAYS_TO_RTW")
rtw_rmse = rtw_mse ** 0.5

rtw_metrics = {"mae": round(rtw_mae, 2), "rmse": round(rtw_rmse, 2)}
print(f"Return-to-work model metrics: {rtw_metrics}")

# Register model
reg.log_model(
    model=rtw_pipeline,
    model_name="RTW_TIMELINE_PREDICTOR",
    version_name="V1",
    comment="Predicts days to return to work using Linear Regression based on injury and worker characteristics",
    metrics=rtw_metrics
)

print("✅ RTW timeline model registered to Model Registry as RTW_TIMELINE_PREDICTOR")


---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
reg.show_models()

# Show versions for each model
print("\nClaim cost model versions:")
reg.get_model("CLAIM_COST_PREDICTOR").show_versions()

print("\nFraud detector versions:")
reg.get_model("FRAUD_DETECTOR").show_versions()

print("\nRTW timeline predictor versions:")
reg.get_model("RTW_TIMELINE_PREDICTOR").show_versions()

print("\n✅ All models registered and ready to add to Intelligence Agent")


---
# Test Model Inference

Test calling each model to make predictions.


In [ ]:
# Test claim cost prediction
cost_model = reg.get_model("CLAIM_COST_PREDICTOR").default
sample_claims = claim_cost_df.limit(5).drop("CLAIM_ID")
cost_preds = cost_model.run(sample_claims, function_name="predict")
print("Claim cost predictions:")
cost_preds.select("INJURY_TYPE", "BODY_PART", "TOTAL_INCURRED", "PREDICTED_COST").show()

# Test fraud detection
fraud_model = reg.get_model("FRAUD_DETECTOR").default
sample_fraud = fraud_df.limit(5).drop("CLAIM_ID")
fraud_preds = fraud_model.run(sample_fraud, function_name="predict")
print("\nFraud risk predictions:")
fraud_preds.select("INJURY_TYPE", "DAYS_TO_REPORT", "IS_FRAUD_RISK", "FRAUD_PREDICTION").show()

# Test return-to-work prediction
rtw_model = reg.get_model("RTW_TIMELINE_PREDICTOR").default
sample_rtw = rtw_df.limit(5).drop("CLAIM_ID")
rtw_preds = rtw_model.run(sample_rtw, function_name="predict")
print("\nReturn-to-work timeline predictions:")
rtw_preds.select("INJURY_TYPE", "BODY_PART", "DAYS_TO_RTW", "PREDICTED_DAYS_TO_RTW").show()

print("\n✅ All models tested successfully!")


---
# Next Steps

## Add Models to Intelligence Agent

1. Execute the wrapper procedures: `sql/ml/07_create_model_wrapper_functions.sql`
2. In Snowsight → AI & ML → Agents → ZENITH_INSURANCE_INTELLIGENCE_AGENT
3. Go to Tools → + Add → Procedure
4. Add each procedure:
   - **PREDICT_CLAIM_COST**
   - **DETECT_FRAUD_RISK**
   - **PREDICT_RTW_TIMELINE**

## Example Questions for Agent

- "Predict claim costs for open back injury claims"
- "Which open claims have high fraud risk according to the fraud detector?"
- "Predict return-to-work timeline for shoulder injuries in healthcare"

The models will now be available as tools your agent can use!
